In [171]:
import numpy as np

In [196]:
class MultiArmed_Bandit:
    def __init__(self, n_arms, context_size, alpha=0.1, learning_rate=0.01):
        self.context_size = context_size  # Cantidad de variables de entrada (d1, d2, d3)
        self.n_arms = n_arms  # Posibles acciones
        self.alpha = alpha    # Probabilidad de exploración (ε)
        self.learning_rate = learning_rate # Tasa de aprendizaje para actualizar los pesos

        # Inicializamos los pesos para cada brazo.
        # Cada brazo tendrá un vector de pesos de tamaño context_size.
        # Además, añadimos un peso para el sesgo (bias) si es necesario,
        # lo que implica que el context_size efectivo para los pesos será context_size + 1
        # si incluimos un 1 constante en el contexto.
        # Para simplificar aquí, asumimos que el contexto ya incluye el término de sesgo si se desea.
        self.weights = np.zeros((self.n_arms, self.context_size))

    def select_arm(self, context):
        if np.random.rand() < self.alpha:  # Exploración: acción aleatoria
            return np.random.randint(0, self.n_arms)
        else:  # Explotación: mejor acción según el modelo
            # Asegurarse de que el contexto sea un array numpy y tenga la forma correcta
            context_np = np.array(context).reshape(1, -1) # Convertir a fila vector

            # Calcular la recompensa esperada para cada brazo
            # La recompensa esperada para cada brazo es el producto punto de su vector de pesos y el contexto.
            expected_rewards = np.dot(self.weights, context_np.T).flatten()

            print(f'Recompensas esperadas para cada brazo: {expected_rewards}')

            exp_rewards = np.exp(expected_rewards - np.max(expected_rewards))
            probabilities = exp_rewards / np.sum(exp_rewards)

            print(f'Probabilidad de coger cada brazo: {probabilities}')

            # Seleccionar un brazo aleatoriamente basado en estas probabilidades
            # np.random.choice permite elegir un elemento de una lista
            # con probabilidades especificadas.
            return np.random.choice(self.n_arms, p=probabilities)
        
    def decode_action(self, chosen_arm, parameters=['f1', 'f2', 'f3'], k_values=[1, 2, 3, 4, 5]):
        param_idx = chosen_arm // len(k_values)
        k_idx = chosen_arm % len(k_values)
        return parameters[param_idx], k_values[k_idx]

    def update(self, chosen_arm, context,reward):
        # Asegurarse de que el contexto sea un array numpy
        context_np = np.array(context)

        print(f'Pesos antes: {self.weights[chosen_arm]}')

        # Actualizar los pesos del brazo elegido.
        # Multiplicamos la recompensa directamente por el contexto y la tasa de aprendizaje.
        # Una recompensa positiva y un contexto dado harán que los pesos se ajusten
        # para favorecer ese brazo en contextos similares.
        # Una recompensa negativa (o baja) hará que los pesos se ajusten en la dirección opuesta,
        # desfavoreciendo ese brazo en contextos similares.
        self.weights[chosen_arm] += self.learning_rate * reward * context_np

        print(f'Pesos después: {self.weights[chosen_arm]}')

In [197]:
k=5
context_size=3
n_arms = context_size*k

In [198]:
MAB = MultiArmed_Bandit(n_arms, context_size)

In [211]:
context=(20,100,50)

In [212]:
chosen_arm= MAB.select_arm(context)

Recompensas esperadas para cada brazo: [0.  0.  0.  0.  0.  0.  0.  0.  4.3 0.  0.  0.  0.  0.  0. ]
Probabilidad de coger cada brazo: [0.01140254 0.01140254 0.01140254 0.01140254 0.01140254 0.01140254
 0.01140254 0.01140254 0.8403645  0.01140254 0.01140254 0.01140254
 0.01140254 0.01140254 0.01140254]


In [213]:
funcion, n_veces = MAB.decode_action(chosen_arm)

In [214]:
print(f'Ahora mejoramos la funcion {funcion} {n_veces} veces, pongamos de ejemplo que el resultado es 400, 230, 27.')

Ahora mejoramos la funcion f2 4 veces, pongamos de ejemplo que el resultado es 400, 230, 27.


In [215]:
valor_sol_prev=2.5
valor_sol_actual=2.6

reward=(valor_sol_actual-valor_sol_prev)/3

In [216]:
MAB.update(chosen_arm, context,reward)

Pesos antes: [0.00666667 0.03333333 0.01666667]
Pesos después: [0.01333333 0.06666667 0.03333333]


# Comparamos la solución previa con el frente de pareto
 la solución actual con el frente de pareto, ponderamos la dierencia de cada f entre 0 y 1 (cuanto tendría que mejorar d1 para que fuera solución, cuanto d2 para que fuera solución y cuánto d3)

Por ejemplo, solución previa 450, 220, 17. si el 450 fuera 437, sería solución. entonces 1-(437/450). lo mismo con el resto.
La nueva, sacamos lo mismo, 400, 230 y 27. si el 400 fuera 397, sería solución. (397/400)+...

La recompensa será (valor_solucion_actual-valor_solucion_previa)/3. Si se acerca mucho a soluciones, el valor de recompensa será alto.

Si saca una solución exacta del frente de pareto, finalizar y dar un -x?

Si saca una solución nueva, recompensar y dar un +x